In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# Transformations applied on each image
transform = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.Normalize((0.5,), (0.5,))])

# Loading the training dataset
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Loading the test dataset
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


In [3]:
class MNet(nn.Module):
    def __init__(self):
        super(MNet, self).__init__()
        # First layer: 784 input features (28x28 pixels), 128 output features
        self.fc1 = nn.Linear(784, 128)
        
        # Second layer: 128 input features, 64 output features
        self.fc2 = nn.Linear(128, 64)
        
        # Output layer: 64 input features, 10 output features (for 10 classes)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        # Flatten the image
        x = x.view(-1, 28*28)
        
        # Apply ReLU activation function after each hidden layer
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        # No activation function needed for the output layer
        x = self.fc3(x)
        
        # Applying log_softmax for classification purposes
        return F.log_softmax(x, dim=1)

In [3]:
class MNet(nn.Module):
    def __init__(self):
        super(MNet, self).__init__()
        # First layer: 784 input features (28x28 pixels), 128 output features
        self.fc1 = nn.Linear(784, 128)
        
        # Second layer: 128 input features, 64 output features
        self.fc2 = nn.Linear(128, 64)
        
        # Output layer: 64 input features, 10 output features (for 10 classes)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        # Flatten the image
        x = x.view(-1, 28*28)
        
        # Apply ReLU activation function after each hidden layer
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        # No activation function needed for the output layer
        x = self.fc3(x)
        
        # Applying log_softmax for classification purposes
        return F.log_softmax(x, dim=1)

In [4]:
net = MNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [5]:
num_epochs = 20

with open("run_log.txt", "w") as log_file:
    for epoch in range(num_epochs):  
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
    
            optimizer.zero_grad()
    
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
            if i % 100 == 99:    
                log_message = f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100:.3f}\n'
                print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100:.3f}')
                log_file.write(log_message)
                running_loss = 0.0

print('Finished Training')


Epoch 1, Batch 100, Loss: 1.053
Epoch 1, Batch 200, Loss: 0.435
Epoch 1, Batch 300, Loss: 0.381
Epoch 1, Batch 400, Loss: 0.358
Epoch 1, Batch 500, Loss: 0.325
Epoch 1, Batch 600, Loss: 0.283
Epoch 1, Batch 700, Loss: 0.282
Epoch 1, Batch 800, Loss: 0.266
Epoch 1, Batch 900, Loss: 0.253
Epoch 2, Batch 100, Loss: 0.229
Epoch 2, Batch 200, Loss: 0.201
Epoch 2, Batch 300, Loss: 0.202
Epoch 2, Batch 400, Loss: 0.188
Epoch 2, Batch 500, Loss: 0.168
Epoch 2, Batch 600, Loss: 0.188
Epoch 2, Batch 700, Loss: 0.186
Epoch 2, Batch 800, Loss: 0.175
Epoch 2, Batch 900, Loss: 0.149
Epoch 3, Batch 100, Loss: 0.144
Epoch 3, Batch 200, Loss: 0.136
Epoch 3, Batch 300, Loss: 0.139
Epoch 3, Batch 400, Loss: 0.152
Epoch 3, Batch 500, Loss: 0.140
Epoch 3, Batch 600, Loss: 0.136
Epoch 3, Batch 700, Loss: 0.121
Epoch 3, Batch 800, Loss: 0.124
Epoch 3, Batch 900, Loss: 0.136
Epoch 4, Batch 100, Loss: 0.098
Epoch 4, Batch 200, Loss: 0.107
Epoch 4, Batch 300, Loss: 0.111
Epoch 4, Batch 400, Loss: 0.108
Epoch 4,

In [6]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
with open('training_log.txt', 'a') as f:
    accuracy_message = f'Accuracy of the network on the 10000 test images with Adam optimizer and 20 epochs: {accuracy:.2f} %\n'
    print(accuracy_message)
    f.write(accuracy_message)


Accuracy of the network on the 10000 test images with Adam optimizer and 20 epochs: 97.71 %

